https://huggingface.co/docs/transformers/tasks/language_modeling
Link to follow the tutorial

In [1]:
import pandas as pd
import numpy as np
import gc
from transformers import Adafactor
import tqdm
from tqdm.notebook import tqdm_notebook
import torch
import evaluate
import wandb
import math

In [2]:
wandb.init(project='LED', resume=True)

wandb: Currently logged in as: fayz. Use `wandb login --relogin` to force relogin


In [ ]:
wandb.login()

In [3]:
if torch.cuda.is_available():
   dev = torch.device("cuda:0")
   print("Running on the GPU")
else:
   dev = torch.device("cpu")
   print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_json('/home/beyond-data/Documents/Faiza/Notebooks for Finetuning/Data/train.json')
test = pd.read_json('/home/beyond-data/Documents/Faiza/Notebooks for Finetuning/Data/test.json')
result_df = pd.DataFrame(columns = ['epoch','training_loss','validation_loss','perplexity','rouge1','rouge2','rougeL','rougeLsum'])

epoch = len(result_df['epoch'])+1

train['input'] = train['Question'] + train['document']
test['input'] = test['Question'] + test['document']
del train['Question']
del test['Question']
del train['document']
del test['document']
#trsin
for i in tqdm_notebook(range(len(train['input']))):
  if len(train['input'][i]) > 4096:
    train['input'][i] = train['input'][i][:4096]
for i in tqdm_notebook(range(len(train['Answer']))):
  if len(train['Answer'][i]) > 512:
    train['Answer'][i] = train['Answer'][i][:512]
#test
for i in tqdm_notebook(range(len(test['input']))):
  if len(test['input'][i]) > 4096:
    test['input'][i] = test['input'][i][:4096]
for i in tqdm_notebook(range(len(test['Answer']))):
  if len(test['Answer'][i]) > 512:
    test['Answer'][i] = test['Answer'][i][:512]

  0%|          | 0/46753 [00:00<?, ?it/s]

  0%|          | 0/46753 [00:00<?, ?it/s]

  0%|          | 0/6676 [00:00<?, ?it/s]

  0%|          | 0/6676 [00:00<?, ?it/s]

In [5]:
epoch = len(result_df['epoch'])

In [6]:
epoch

0

In [7]:
from transformers import LEDTokenizer, LEDForConditionalGeneration
tokenizer = LEDTokenizer.from_pretrained('allenai/led-base-16384')
if len(result_df['epoch'])>0:
    model = LEDForConditionalGeneration.from_pretrained("LEDforQAS"+str(epoch+1)).to(dev)
else:
    model = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384").to(dev)

seed = 1337
np.random.seed(seed)
#random.seed(52)
train_df=train
batch_size=3
num_of_batches=len(train_df)/batch_size
print(num_of_batches)
gc.collect()

optimizer = Adafactor(model.parameters(),lr=0.001,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

from IPython.display import HTML, display
def progress(loss,value, max=100):
 return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

15584.333333333334


### Saved fine-tuned model

In [8]:
optimizer = Adafactor(model.parameters(),lr=2e-5,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

from IPython.display import HTML, display
def progress(loss,value, max=100):
 return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [9]:
def model_validation(model,epoch, running_loss):
    model.eval()
    bs = 1
    val_loss=0
    ppl=0
    test_num_of_batches=len(test)/bs
    predictions = []
    torch.cuda.empty_cache()
    gc.collect()
    print("Validating the model")
    for i in tqdm_notebook(range(int(test_num_of_batches))):
        ib=[]
        lb = []
        new_df=test[i*bs:i*bs+bs]
        for indx,row in new_df.iterrows():
          input =row['input']
          ib.append(input)
          lb.append(row['Answer'])
          torch.cuda.empty_cache()
        ib=tokenizer.batch_encode_plus(ib,truncation=True,padding=True,max_length=4096,return_tensors='pt')["input_ids"].to(dev)
        lb=tokenizer.batch_encode_plus(lb,truncation=True,padding=True,max_length=512,return_tensors='pt')["input_ids"].to(dev)
        optimizer.zero_grad()
        outputs = model(input_ids=ib,labels=lb)
        loss = outputs.loss
        val_loss += loss.item()
    val_loss = val_loss/test_num_of_batches
    print("Validation loss: ", val_loss)
    wandb.log({"Training loss":running_loss,"Validation loss": val_loss})
    try:
        val_loss = np.mean(val_loss)
        ppl = math.exp(val_loss)
        wandb.log({"Perplexity": ppl})
    except:
        pass
    result_row = []
    result_row.append(epoch)
    result_row.append(running_loss)
    result_row.append(val_loss)
    result_row.append(ppl)
    result_row.append(0)
    result_row.append(0)
    result_row.append(0)
    result_row.append(0)
    result_df.loc[len(result_df)]=result_row
    result_df.to_csv('LEDresults.csv')

In [10]:
#from os import truncate
model.train()

loss_per_10_steps=[]
num_of_epochs=30
for epoch in range(1,num_of_epochs+1):
  print('Running epoch: {}'.format(epoch))
  
  running_loss=0

  out = display(progress(1, num_of_batches+1), display_id=True)
  for i in tqdm_notebook(range(int(num_of_batches))):
    inputbatch=[]
    labelbatch=[]
    new_df=train_df[i*batch_size:i*batch_size+batch_size]
    for indx,row in new_df.iterrows():
      input = 'LFQA: '+row['input']+'</s>' 
      labels = row['Answer']+'</s>'   
      inputbatch.append(input)
      labelbatch.append(labels)
      torch.cuda.empty_cache()
    inputbatch=tokenizer.batch_encode_plus(inputbatch,truncation=True,padding=True,max_length=512,return_tensors='pt')["input_ids"]
    labelbatch=tokenizer.batch_encode_plus(labelbatch,truncation = True,padding=True,max_length=512,return_tensors="pt") ["input_ids"]
    inputbatch=inputbatch.to(dev)
    labelbatch=labelbatch.to(dev)

    # clear out the gradients of all Variables 
    optimizer.zero_grad()

    # Forward propogation
    outputs = model(input_ids=inputbatch, labels=labelbatch)
    loss = outputs.loss
    loss_num=loss.item()
    logits = outputs.logits
    running_loss+=loss_num
    out.update(progress(loss_num,i, num_of_batches+1))

    # calculating the gradients
    loss.backward()

    #updating the params
    optimizer.step()
    #torch.empty_cache()
    gc.collect()
  running_loss=running_loss/int(num_of_batches)
  print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))
  #wandb.log({"Training loss": running_loss})
  wandb.watch(model)
  model.save_pretrained('LEDforQAS'+str(epoch))
  model_validation(model, epoch, running_loss)

Running epoch: 1


  0%|          | 0/15584 [00:00<?, ?it/s]

wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error resolved after 0:00:08.772819, resuming normal operation.


Epoch: 1 , Running loss: 1.8960357526990392
Validating the model


  0%|          | 0/6676 [00:00<?, ?it/s]

...

  0%|          | 0/15584 [00:00<?, ?it/s]

wandb: Network error (ConnectionError), entering retry loop.


Epoch: 30 , Running loss: 0.0348857226791853
Validating the model


  0%|          | 0/6676 [00:00<?, ?it/s]

Validation loss:  5.610978714635412


In [8]:
result_df

,epoch,training_loss,validation_loss,perplexity,rouge1,rouge2,rougeL,rougeLsum


In [26]:
val_loss = 2.637076

In [27]:
val_loss = np.mean(val_loss)
ppl = math.exp(val_loss)
wandb.log({"Perplexity": ppl})

In [28]:
ppl

13.972288846915914

In [22]:
result_df['perplexity'][0] = ppl

In [23]:
result_df.to_csv('LEDresults.csv')

In [14]:
gc.collect()
torch.cuda.empty_cache()
model_validation(model, epoch, 1.8947438496371514)

Validating the model


  0%|          | 0/6676 [00:00<?, ?it/s]

Validation loss:  4.6370755106907104


In [ ]:
def generate_answers(batch):
    #print(batch)
    inputs_dict = tokenizer(
        batch[0], max_length=16384, padding="max_length", truncation=True, return_tensors="pt"
    )
    input_ids = inputs_dict.input_ids.to(dev)
    attention_mask = inputs_dict.attention_mask.to(dev)
    output_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=512, num_beams=2)
    result = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    return result


#result = dataset.map(generate_answer, batched=True, batch_size=2)
#rouge = evaluate.load("rouge")
#rouge.compute(predictions=result["predicted_abstract"], references=result["abstract"])